In [1]:
import xarray as xr
import os
import glob
import numpy as np
import pandas as pd
import dask.dataframe as dd
from distributed import Client

/glade/u/home/mortimer/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/glade/u/home/mortimer/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


https://github.com/pydata/xarray/pull/2706

In [19]:
files = glob.glob('../../argo/**/*_prof.nc',recursive=True)

In [20]:
pd.DataFrame(files,columns=['files']).sort_values('files').to_csv('../../argo/catalogue.csv',index=None)

In [21]:
dffiles = pd.read_csv('../../argo/catalogue.csv')

In [23]:
files = list(dffiles.files)


In [4]:
claient = Client()
client

Client Scheduler: tcp://127.0.0.1:47688 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 9 Cores: 72 Memory: 270.64 GB


In [155]:
from functools import partial

data_types ={'CONFIG_MISSION_NUMBER':'float64','CYCLE_NUMBER':'float64','DATA_CENTRE':'|S2','DATA_MODE':'|S1',
             'DATA_STATE_INDICATOR':'|S4','DC_REFERENCE':'|S32','DIRECTION':'|S1','FIRMWARE_VERSION':'|S32',
             'FLOAT_SERIAL_NO':'|S32','JULD':'float64','JULD_LOCATION':'float64','JULD_QC':'|S1','LATITUDE':'float64',
             'LONGITUDE':'float64','PI_NAME':'|S64','PLATFORM_NUMBER':'|S8','PLATFORM_TYPE':'|S32','POSITIONING_SYSTEM':'|S8',
             'POSITION_QC':'|S1','PRES':'float64','PRES_ADJUSTED':'float64','PRES_ADJUSTED_ERROR':'float64',
             'PRES_ADJUSTED_QC':'|S1','PRES_QC':'|S1','PROFILE_PRES_QC':'|S1','PROFILE_PSAL_QC':'|S1','PROFILE_TEMP_QC':'|S1',
             'PROJECT_NAME':'|S64','PSAL':'float64','PSAL_ADJUSTED':'float64','PSAL_ADJUSTED_ERROR':'float64',
             'PSAL_ADJUSTED_QC':'|S1','PSAL_QC':'|S1','TEMP':'float64','TEMP_ADJUSTED':'float64','TEMP_ADJUSTED_ERROR':'float64',
             'TEMP_ADJUSTED_QC':'|S1','TEMP_QC':'|S1','VERTICAL_SAMPLING_SCHEME':'|S256','WMO_INST_TYPE':'|S4'}
def process_mf(dsinput,levels,data_types=data_types):
    ds = xr.Dataset()
    pad =xr.DataArray(np.ones((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))) *np.nan,dims={'N_PROF','N_LEVELS'})
    pad_qc = xr.DataArray(np.chararray((len(dsinput.N_PROF),levels-len( dsinput.N_LEVELS))),dims={'N_PROF','N_LEVELS'})
    pad_qc[:] = b'0'
    pading =xr.DataArray(pad,dims={'N_PROF','N_LEVELS'})
    for varname in data_types.keys():
        if varname in dsinput.data_vars:
            da = dsinput[varname]
            if 'N_LEVELS' in da.dims:   
                if varname in dsinput.data_vars:
                    if varname.endswith('QC'):
                        da = xr.concat([dsinput[varname],pad_qc],dim='N_LEVELS').astype(data_types[varname])
                    else:
                        da = xr.concat([dsinput[varname],pading],dim='N_LEVELS').astype(data_types[varname])
            else:
                da = dsinput[varname].astype(data_types[varname])
        else:
            if data_types[varname]=='float64':
                da = xr.DataArray(np.ones(len(dsinput.N_PROF))*np.nan , name=varname, dims=['N_PROF'])
            else:
                da = xr.DataArray(np.chararray((len(dsinput.N_PROF))), name=varname, dims=['N_PROF'])
        if not ('HISTORY' in varname) and ('N_CALIB' not in da.dims) and ('N_PARAM' not in da.dims) and  ('N_PROF' in da.dims):
                ds[varname]= da
    return ds

template =xr.open_dataset(files[200])    
preproc = partial(process_mf,levels=2000)


In [161]:
%%time
import zarr
import os
data =  preproc(xr.open_dataset(files[200]))
data =data.chunk({'N_PROF':10000,'N_LEVELS':2000})

zarr_path =  '../../argozarr/argo.zarr' 
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
encoding = {vname: {'compressor': compressor} for vname in data.variables}
data.to_zarr(zarr_path,mode='w',encoding=encoding)

zarr_dst = zarr.open(zarr_path, mode='a')
for file in files[100:150]:
    data =  preproc(xr.open_dataset(file))
    data =data.chunk({'N_PROF':100,'N_LEVELS':1000})
    zarr_path_temp =  '../../argo_temp/argo.zarr' 
    compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
    encoding = {vname: {'compressor': compressor} for vname in data.variables}
    data.to_zarr(zarr_path_temp,mode='w',encoding=encoding)
    zarr_src = zarr.open(zarr_path_temp, mode='r')
    for key in [k for k in zarr_src.array_keys()]:
        zarr_dst[key].append(zarr_src[key])


CPU times: user 1min 3s, sys: 9.28 s, total: 1min 12s
Wall time: 1min 6s


In [163]:
%%time
data =xr.open_mfdataset(files[:150],preprocess=preproc,concat_dim='N_PROF',chunks={'N_PROF':1000,'N_LEVELS':1000})
data =data.chunk({'N_PROF':10000,'N_LEVELS':2000})
data.to_zarr(zarr_path,mode='w')

CPU times: user 1min 49s, sys: 4.87 s, total: 1min 54s
Wall time: 1min 48s


In [140]:
data =data.chunk({'N_PROF':100,'N_LEVELS':1000})


In [143]:
zarr_path = '../../mfzarr/argo.zarr'

In [144]:
data.to_zarr(zarr_path)

/glade/u/home/mortimer/anaconda3/lib/python3.7/site-packages/xarray/conventions.py:172: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  SerializationWarning)


In [165]:
%%time
for file in files[0:100]:
    #with dask.config.set({'array.chunk-size': '2 MiB'}):
    ds1 = preproc(xr.open_dataset(file, chunks=10),)



CPU times: user 24.7 s, sys: 48 ms, total: 24.8 s
Wall time: 26.1 s


In [18]:
ds1 = xr.open_zarr(zarr_path)
ds1.info()

xarray.Dataset {
dimensions:
	N_LEVELS = 2000 ;
	N_PROF = 11988 ;

variables:
	float64 CONFIG_MISSION_NUMBER(N_PROF) ;
		CONFIG_MISSION_NUMBER:conventions = 1...N, 1 : first complete mission ;
		CONFIG_MISSION_NUMBER:long_name = Unique number denoting the missions performed by the float ;
	float64 CYCLE_NUMBER(N_PROF) ;
		CYCLE_NUMBER:conventions = 0...N, 0 : launch cycle (if exists), 1 : first complete cycle ;
		CYCLE_NUMBER:long_name = Float cycle number ;
	|S2 DATA_CENTRE(N_PROF) ;
		DATA_CENTRE:conventions = Argo reference table 4 ;
		DATA_CENTRE:long_name = Data centre in charge of float data processing ;
	|S1 DATA_MODE(N_PROF) ;
		DATA_MODE:conventions = R : real time; D : delayed mode; A : real time with adjustment ;
		DATA_MODE:long_name = Delayed mode or real time data ;
	|S4 DATA_STATE_INDICATOR(N_PROF) ;
		DATA_STATE_INDICATOR:conventions = Argo reference table 6 ;
		DATA_STATE_INDICATOR:long_name = Degree of processing the data have passed through ;
	|S16 DATA_TYPE(N_PROF) 

In [63]:
for key in [k for k in zarr_src.array_keys()]:
    print('\'%s\':\'%s\',' % (key,zarr_dst[key].dtype))

'CONFIG_MISSION_NUMBER':'float64',
'CYCLE_NUMBER':'float64',
'DATA_CENTRE':'|S2',
'DATA_MODE':'|S1',
'DATA_STATE_INDICATOR':'|S4',
'DC_REFERENCE':'|S32',
'DIRECTION':'|S1',
'FIRMWARE_VERSION':'|S32',
'FLOAT_SERIAL_NO':'|S32',
'JULD':'float64',
'JULD_LOCATION':'float64',
'JULD_QC':'|S1',
'LATITUDE':'float64',
'LONGITUDE':'float64',
'PI_NAME':'|S64',
'PLATFORM_NUMBER':'|S8',
'PLATFORM_TYPE':'|S32',
'POSITIONING_SYSTEM':'|S8',
'POSITION_QC':'|S1',
'PRES':'float64',
'PRES_ADJUSTED':'float64',
'PRES_ADJUSTED_ERROR':'float64',
'PRES_ADJUSTED_QC':'|S1',
'PRES_QC':'|S1',
'PROFILE_PRES_QC':'|S1',
'PROFILE_PSAL_QC':'|S1',
'PROFILE_TEMP_QC':'|S1',
'PROJECT_NAME':'|S64',
'PSAL':'float64',
'PSAL_ADJUSTED':'float64',
'PSAL_ADJUSTED_ERROR':'float64',
'PSAL_ADJUSTED_QC':'|S1',
'PSAL_QC':'|S1',
'TEMP':'float64',
'TEMP_ADJUSTED':'float64',
'TEMP_ADJUSTED_ERROR':'float64',
'TEMP_ADJUSTED_QC':'|S1',
'TEMP_QC':'|S1',
'VERTICAL_SAMPLING_SCHEME':'|S256',
'WMO_INST_TYPE':'|S4',
